## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Activation, Dropout

## 下載資料   

[CIFAR10/CIFAR100 數據集介紹](https://www.itread01.com/content/1519212167.html)

In [2]:
( x_train, y_train ), ( x_test, y_test ) = cifar10.load_data()

print( f'Shape of Training Data = {x_train.shape}' ) 

Shape of Training Data = (50000, 32, 32, 3)


## 資料正規化

In [3]:
## Normalize Data
def normalize( X_train, X_test ):
    mean = np.mean( X_train, axis=(0,1,2,3) )
    std = np.std( X_train, axis=(0,1,2,3) )
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7) 
    return X_train, X_test, mean, std
    
## Normalize Training and Testset    
x_train, x_test, mean_train, std_train = normalize( x_train, x_test ) 

## 獨熱編碼

In [4]:
''' 
OneHot Label 由 ( None, 1 ) -> ( None, 10 )
例如 label=2 轉變成 [0,0,1,0,0,0,0,0,0,0]
'''
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform( y_train ).toarray()
y_test = one_hot.transform( y_test ).toarray()

## 建立 CNN 模型

In [5]:
classifier = Sequential( name='CNN_Model' )

# 卷積組合
classifier.add( Conv2D( 32, kernel_size=(3,3), padding='same', input_shape=(32,32,3), activation='relu', name='Convolution_1' ) )
classifier.add( BatchNormalization( name='BN_1' ) )

# 池化
classifier.add( MaxPooling2D( pool_size=(2,2), name='Max_Pooling' ) )

# 卷積組合
classifier.add( Conv2D( 64, kernel_size=(3,3), padding='same', activation='relu', name='Convolution_2' ) )
classifier.add( BatchNormalization( name='BN_2' ) )

# Flatten
classifier.add( Flatten( name='Flatten' ) )

# Fully Connected
classifier.add( Dense( 100, activation='relu', name='FC' )) 

# 輸出
classifier.add( Dense( 10, activation='softmax', name='Softmax' ) )

classifier.summary( )

Model: "CNN_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Convolution_1 (Conv2D)       (None, 32, 32, 32)        896       
_________________________________________________________________
BN_1 (BatchNormalization)    (None, 32, 32, 32)        128       
_________________________________________________________________
Max_Pooling (MaxPooling2D)   (None, 16, 16, 32)        0         
_________________________________________________________________
Convolution_2 (Conv2D)       (None, 16, 16, 64)        18496     
_________________________________________________________________
BN_2 (BatchNormalization)    (None, 16, 16, 64)        256       
_________________________________________________________________
Flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
FC (Dense)                   (None, 100)               16

## 訓練模型

In [6]:
#超過兩個就要選categorical_crossentrophy
classifier.compile( optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )
classifier.fit( x_train, y_train, batch_size=100, epochs=20 )

Train on 50000 samples
Epoch 1/20
50000/50000 [==============================] - 5s 105us/sample - loss: 1.3260 - accuracy: 0.5389
Epoch 2/20
50000/50000 [==============================] - 3s 68us/sample - loss: 0.8190 - accuracy: 0.7128
Epoch 3/20
50000/50000 [==============================] - 3s 68us/sample - loss: 0.5678 - accuracy: 0.8029
Epoch 4/20
50000/50000 [==============================] - 3s 68us/sample - loss: 0.3718 - accuracy: 0.8716
Epoch 5/20
50000/50000 [==============================] - 3s 68us/sample - loss: 0.2302 - accuracy: 0.9219
Epoch 6/20
50000/50000 [==============================] - 3s 68us/sample - loss: 0.1526 - accuracy: 0.9493
Epoch 7/20
50000/50000 [==============================] - 3s 67us/sample - loss: 0.1267 - accuracy: 0.9561
Epoch 8/20
50000/50000 [==============================] - 3s 70us/sample - loss: 0.1117 - accuracy: 0.9619
Epoch 9/20
50000/50000 [==============================] - 4s 71us/sample - loss: 0.1019 - accuracy: 0.9648
Epoch 10/20
5

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=( np.zeros(shape=(1,32,32,3))-mean_train )/( std_train+1e-7 ) 
classifier.predict( input_example )

array([[9.6832943e-04, 7.9826656e-10, 2.4272746e-03, 9.9658352e-01,
        1.4162015e-07, 8.0282987e-08, 1.6436826e-09, 2.3032469e-07,
        2.0442099e-05, 7.3169866e-13]], dtype=float32)